<a href="https://colab.research.google.com/github/TengFwu/gemini_alura/blob/main/Julius_trip_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display

In [ ]:
# Instala os pacotes necessários
%pip -q install google-genai google-adk

In [ ]:
# Importações essenciais
import os
import textwrap
import warnings
from IPython.display import display, Markdown
from datetime import date
from google.colab import userdata
from google import genai
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

In [38]:
# Configuração da API Key do Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY') # coloque a sua chave API


warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar: executa a conversa com qualquer agente e retorna a resposta final
def executar_agente(agent: Agent, mensagem: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=mensagem)])

    resposta_final = ""
    for evento in runner.run(user_id="user1", session_id="session1", new_message=content):
        if evento.is_final_response():
            for parte in evento.content.parts:
                if parte.text:
                    resposta_final += parte.text + "\n"
    return resposta_final

# Função auxiliar: formata texto como Markdown no Colab
def mostrar_markdown(texto):
    texto_formatado = texto.replace('•', '  *')  # Corrige bullet points
    return Markdown(textwrap.indent(texto_formatado, '> ', predicate=lambda _: True))

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [41]:
import google.generativeai as genai
# Inicializa o modelo Gemini Pro com chat e histórico vazio
agente_descoberta_viagem = genai.GenerativeModel('gemini-2.0-flash').start_chat(history=[])

# Função para interação com o primeiro agente (descoberta de perfil e sugestão de destinos)
def obter_destinos_via_perfil():
    print("👋 Olá! Vamos planejar sua próxima viagem juntos!\n")

    # Coleta de informações do usuário
    grupo = input("👥 Quem está viajando? (Ex: Só eu, com meu cônjuge, com a família, com amigos)\n")
    experiencia = input("🌄 Qual o tipo de experiência que você busca? (Ex: Relaxamento total, aventura, vida noturna agitada, contato com a natureza, etc.)\n")
    origem = input("📍 De onde você está partindo? (Qual a sua cidade/estado de origem?)\n")
    duracao = input("📆 Quantos dias você pretende ficar?\n")
    preferencias = input("📝 Você tem alguma preferência ou restrição específica?\n")
    valor_proposto = input("💰 Quanto você pretende gastar para viajar por pessoa (desconsiderando translado)?\n")

    # Cria o perfil textual do usuário com base nas respostas
    perfil_usuario = (
        f"Quem está viajando: {grupo}\n"
        f"Tipo de experiência buscada: {experiencia}\n"
        f"Origem da viagem: {origem}\n"
        f"Duração da viagem: {duracao} dias\n"
        f"Preferências e restrições: {preferencias}\n"
        f"Valor Proposto: {valor_proposto}"
    )

    print("\n🔍 Procurando destinos ideais com base no seu perfil...\n")

    # Envia a solicitação ao modelo Gemini com instrução clara para retornar apenas uma lista de destinos
    resposta = agente_descoberta_viagem.send_message(
        perfil_usuario +
        "\n\nCom base nessas informações, sugira de 10 destinos ideais para essa viagem. " +
        "\n\nForneça destinos nacionais e internacionais. " +
        "Retorne a resposta diretamente como uma lista com bullet points, " +
        "sem pedir mais informações ao usuário. Não inclua explicações nem introduções. " +
        "Apenas a lista dos destinos sugeridos."
    )

    # Extrai e retorna o texto da resposta (lista de destinos)
    destinos_sugeridos = resposta.text.strip()

    print("🏝️ Destinos sugeridos:\n")
    print(destinos_sugeridos)

    return destinos_sugeridos

In [49]:
def agente_custos_destino(lista_destinos: str):
    agente = Agent(
        name="avaliador_custos_destino",
        model="gemini-2.0-flash",
        instruction="""
        Receba uma lista de destinos e estime o gasto médio por pessoa por dia (excluindo transporte principal e hospedagem).
        Deve incluir: alimentação, transporte local e atividades básicas.
        Use USD como base e converta para a moeda local do usuário.
        Também procure qual a melhor época para viajar em cada destino e os pontos turísticos mais famosos.
        """,
        description="Estima custos médios por dia no destino e melhores perídos e locais para conhecer",
        tools=[google_search]
    )
    return executar_agente(agente, lista_destinos)


In [ ]:
def agente_precos_voos(descricao_viagem: str):
    agente = Agent(
        name="estimador_voos",
        model="gemini-2.0-flash",
        instruction="""
        Receba origem, destino e período da viagem. Estime o custo médio de ida e volta em BRL ou USD.
        Procure dados de voos utilizando o google.flight
        """,
        description="Estima preços de voos e fornece links úteis",
        tools=[google_search]
    )
    return executar_agente(agente, descricao_viagem)


In [48]:
def agente_comparador_tabela(entrada_dados: str):
    agente = Agent(
        name="tabela_comparativa_destinos",
        model="gemini-2.0-flash",
        instruction="""
        Receba os dados de custos dos destinos, voos, melhores períodos de viagem e locais turísticos locais e apresente uma tabela comparativa com todos os destinos fornecidos.
        A tabela deve ajudar o usuário a escolher o melhor custo-benefício. Valores não presentes, coloque NA
        """,
        description="Cria tabela comparativa de destinos"
    )
    return executar_agente(agente, entrada_dados)



In [47]:
def agente_detalhes_local(local_escolhido: str):
    agente = Agent(
        name="guia_local_cultural",
        model="gemini-2.0-flash",
        instruction="""
        Descreva a cultura do local, pontos turísticos imperdíveis, pratos típicos e links de restaurantes muito bem avaliados.
        Formate a resposta em tabela.
        Caso a pessoa escolha dois locais, gere uma tabela comparativa e termine com um pequeno texto sumarizando a decisão com os pontos fortes de cada uma
        """,
        description="Resumo do destino com foco em turismo e gastronomia",
        tools=[google_search]
    )
    return executar_agente(agente, local_escolhido)



In [53]:
def planejar_viagem_interativa():
    print("👋 Olá! Vamos planejar sua próxima viagem juntos!\n")

    # 🔹 Coletar respostas do usuário
    quem_vai = input("👥 Quem está viajando? (Ex: Só eu, com meu cônjuge, com a família, com amigos)\n")
    tipo_experiencia = input("🌄 Qual o tipo de experiência que você busca? (Ex: Relaxamento total, aventura, vida noturna agitada, contato com a natureza, etc.)\n")
    origem = input("📍 De onde você está partindo? (Qual a sua cidade/estado de origem?)\n")
    duracao = input("📆 Quantos dias você pretende ficar?\n")
    preferencias = input("📝 Você tem alguma preferência ou restrição específica? (Ex: Quero conhecer a China\n")
    valor_proposto = input("💰 Quanto você pretende gastar para viajar por pessoa (desconsiderando translado)?\n")

    # 🔹 Montar perfil completo
    perfil_usuario = f"""
    Perfil da viagem:
    - Quem vai: {quem_vai}
    - Tipo de experiência: {tipo_experiencia}
    - Origem: {origem}
    - Duração: {duracao} dias
    - Preferências/Restrições: {preferencias}
    - Valor Proposto: {valor_proposto}
    """

    # 🔹 Enviar perfil ao primeiro agente

    print("\n🔍 Procurando destinos ideais com base no seu perfil...\n")
    destinos_sugeridos = agente_descoberta_viagem.send_message(
        f"Com base nesse perfil, sugira 5 destinos ideais e justifique cada um:\n{perfil_usuario}"
    ).text

    # 2. Estimar custos diários nos destinos sugeridos
    print("\n💰 Estimando os custos médios por pessoa por dia em cada destino...\n")
    custos_destinos = agente_custos_destino(destinos_sugeridos)

    # 3. Estimar preços de voos
    print("\n🛫 Estimando preços médios de voos a partir da sua origem...\n")
    estimativa_voos = agente_precos_voos(destinos_sugeridos)

    # 4. Criar tabela comparativa
    print("\n📊 Gerando tabela comparativa dos destinos com base nos custos e voos...\n")
    comparativo_input = f"{custos_destinos}\n\n{estimativa_voos}"
    tabela_comparativa = agente_comparador_tabela(comparativo_input)
    display(mostrar_markdown(tabela_comparativa))

    # 5. Escolha do destino
    destino_final = input("🌍 Qual dos destinos você mais gostou? Escolha um ou mais destinos e saiba mais sobre\n")
    print(f"\n📚 Reunindo informações culturais, turísticas e gastronômicas...\n")
    detalhes_destino = agente_detalhes_local(destino_final)
    display(mostrar_markdown(detalhes_destino))

    print("\n🎉 Pronto! Agora você tem tudo para começar a planejar sua viagem com segurança e empolgação!\n")


In [ ]:
planejar_viagem_interativa()


👋 Olá! Vamos planejar sua próxima viagem juntos!

